In [86]:
import gym
import numpy as np

env = gym.make('FrozenLake8x8-v0')

In [87]:
def white(x):
    return '\x1b[0;37;40m' + x + '\x1b[0m'

def green(x):
    return '\x1b[6;30;42m' + x + '\x1b[0m'

def red(x):
    return '\x1b[3;30;41m' + x + '\x1b[0m'

lake_row = [
    "SFFFFFFF",
    "FFFFFFFF",
    "FFFHFFFF",
    "FFFFFHFF",
    "FFFHFFFF",
    "FHHFFFHF",
    "FHFFHFHF",
    "FFFHFFFG"
]
lake = [c for s in lake_row for c in s]

### We want to find the optimal policy (in this case, maximizes cumulative reward as well as minimizes falling into hole risk, due to single positive reward upon reaching goal)

$$\pi^*(s) = \underset{a}{\operatorname{argmax}} \sum_{s'} P(s'|s,a)U(s')$$
$$U(s) = R(s) + \gamma \max_a \sum_{s'} P(s'|s,a)U(s')$$

### Recall, definition is recursive, but can be solved bottom up through iteration: Value Iteration or Policy Iteration. We will implement Policy Iteration

### Functions

In [114]:
'''
Constants we can play around with
'''
DISCOUNT = 1.0
STEP_REWARD = 0.0
LOSE_REWARD = 0.0
WIN_REWARD = 1.0

def avg_reward(env, s, a):
    '''
    Helper function
    '''
    avg_reward = 0
    for prob, next_s, reward, done in env.env.P[s][a]:
        if not done:
            avg_reward += prob * STEP_REWARD
        elif reward == 0.0:
            avg_reward += prob * LOSE_REWARD
        else:
            avg_reward += prob * WIN_REWARD
    return avg_reward
            
def random_policy(env):
    '''
    Helper function
    Returns a random policy:
        each state is mapped to a random action
    '''
    return np.random.randint(0, env.env.nA, size=env.env.nS)

def one_step_lookahead(env, s, value_function):
    '''
    Helper function
    Returns possible actions' value from the current state:
        each action is mapped to its value according to the value_function
    '''
    action_values = np.zeros(env.env.nA)
    for a in range(env.env.nA):
        value = avg_reward(env, s, a)
        for p, next_s, _, _ in env.env.P[s][a]:
            value += DISCOUNT * p * value_function[next_s]
        action_values[a] = value
    return action_values
    
def policy_evaluation(env, policy, max_backups=1000, tol=1e-6):
    '''
    Helper function
    Returns a value_function by following the policy
    '''
    old_value = np.zeros(env.env.nS)
    for i in range(max_backups):
        new_value = np.zeros(env.env.nS)
        for s in range(env.env.nS):
            action_values = one_step_lookahead(env, s, old_value)
            new_value[s] = action_values[policy[s]]
        if np.max(np.abs(new_value-old_value)) < tol:
            break
        old_value = new_value
        
    return new_value

def policy_improvement(env, value_function):
    '''
    Helper function
    Returns a policy by greedily choosing the action which
        has the highest value according to the value_function
    '''
    policy = np.zeros(env.env.nS, dtype=np.int32)
    for s in range(env.env.nS):
        action_values = one_step_lookahead(env, s, value_function)
        policy[s] = np.argmax(action_values)
    return policy

def policy_iteration(env, max_steps=100):
    '''
    Main function
    Returns optimal policy by iteratively performing
        policy evaluation and policy improvement
    '''
    opt_policy = random_policy(env)
    for i in range(max_steps):
        value_function = policy_evaluation(env, opt_policy)
        new_policy = policy_improvement(env, value_function)
        
        if np.array_equal(new_policy, opt_policy):
            print('converged after {} iterations'.format(i))
            break
            
        opt_policy = new_policy
        
    return opt_policy, value_function

### Find our optimal policy

In [115]:
opt_policy, value_func = policy_iteration(env)

converged after 11 iterations


### Visualize our optimal policy and its corresponding value function

In [116]:
assert(len(opt_policy) == env.env.nS)
opt_policy

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 0, 0, 0, 0, 2, 3,
       3, 2, 0, 0, 0, 1, 0, 0, 2, 2, 0, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 2, 1, 0],
      dtype=int32)

In [117]:
actions = ['L','D','R','U']

print('π*: Optimal Policy')
for i,a in enumerate(opt_policy):
    if i and not i % 8:
        print('')
        
    if lake[i] == 'S':
        print(white(actions[a]), end='   ')
    elif lake[i] == 'G':
        print(green('G'), end='   ')
    elif lake[i] == 'H':
        print(red(actions[a]), end='   ')
    else:
        print(actions[a], end='   ')
        
print('\n\nV: Corresponding Value Fuction')
        
for i,v in enumerate(value_func):
    if i and not i % 8:
        print('')
        
    v = str(round(v,1))
        
    if lake[i] == 'S':
        print(white(v), end=' ')
    elif lake[i] == 'G':
        print(green('-G-'), end=' ')
    elif lake[i] == 'H':
        print(red(v), end=' ')
    else:
        print(v, end=' ')

π*: Optimal Policy
U   R   R   R   R   R   R   R   
U   U   U   U   U   U   U   R   
L   L   L   L   R   U   U   R   
L   L   L   D   L   L   R   R   
L   U   L   L   R   D   U   R   
L   L   L   D   U   L   L   R   
L   L   D   L   L   L   L   R   
L   D   L   L   D   R   D   G   

V: Corresponding Value Fuction
1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 
1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 
1.0 1.0 0.9 0.0 0.9 0.9 1.0 1.0 
1.0 0.9 0.8 0.5 0.6 0.0 0.9 1.0 
1.0 0.8 0.5 0.0 0.5 0.6 0.9 1.0 
1.0 0.0 0.0 0.2 0.4 0.4 0.0 1.0 
1.0 0.0 0.2 0.1 0.0 0.3 0.0 1.0 
1.0 0.7 0.5 0.0 0.3 0.6 0.8 -G- 

### Evaluate our optimal policy

In [118]:
rewards = []
for i_episode in range(1000):
    observation = env.reset()
    for t in range(200):
        action = opt_policy[observation]
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode {: >4}: {: >4} after {: >3} timesteps".format(
                i_episode+1, 'Won' if reward else 'Lost', t+1))
            break
    rewards.append(reward)

print('\nAverage reward: {}'.format(np.mean(rewards)))

Episode    1:  Won after  93 timesteps
Episode    2:  Won after  52 timesteps
Episode    3:  Won after 106 timesteps
Episode    4: Lost after 200 timesteps
Episode    5: Lost after 200 timesteps
Episode    6:  Won after  74 timesteps
Episode    7:  Won after 148 timesteps
Episode    8:  Won after  31 timesteps
Episode    9:  Won after 165 timesteps
Episode   10:  Won after 118 timesteps
Episode   11:  Won after 176 timesteps
Episode   12: Lost after 200 timesteps
Episode   13:  Won after  58 timesteps
Episode   14:  Won after 135 timesteps
Episode   15:  Won after 195 timesteps
Episode   16: Lost after 200 timesteps
Episode   17: Lost after 200 timesteps
Episode   18:  Won after  71 timesteps
Episode   19:  Won after 181 timesteps
Episode   20:  Won after 123 timesteps
Episode   21:  Won after  37 timesteps
Episode   22:  Won after 114 timesteps
Episode   23:  Won after 184 timesteps
Episode   24:  Won after 108 timesteps
Episode   25:  Won after 100 timesteps
Episode   26:  Won after 

Episode  224:  Won after  61 timesteps
Episode  225:  Won after 123 timesteps
Episode  226:  Won after 129 timesteps
Episode  227:  Won after 106 timesteps
Episode  228:  Won after 123 timesteps
Episode  229:  Won after  54 timesteps
Episode  230:  Won after 172 timesteps
Episode  231:  Won after 115 timesteps
Episode  232:  Won after 126 timesteps
Episode  233:  Won after  81 timesteps
Episode  234:  Won after 102 timesteps
Episode  235:  Won after  42 timesteps
Episode  236:  Won after  65 timesteps
Episode  237:  Won after  77 timesteps
Episode  238:  Won after  61 timesteps
Episode  239:  Won after  38 timesteps
Episode  240:  Won after 121 timesteps
Episode  241:  Won after  66 timesteps
Episode  242:  Won after 181 timesteps
Episode  243:  Won after 165 timesteps
Episode  244: Lost after 200 timesteps
Episode  245:  Won after  88 timesteps
Episode  246:  Won after  55 timesteps
Episode  247:  Won after  85 timesteps
Episode  248: Lost after 200 timesteps
Episode  249:  Won after 

Episode  457:  Won after  93 timesteps
Episode  458: Lost after 200 timesteps
Episode  459:  Won after  72 timesteps
Episode  460:  Won after 131 timesteps
Episode  461:  Won after  81 timesteps
Episode  462:  Won after 147 timesteps
Episode  463:  Won after  53 timesteps
Episode  464:  Won after 195 timesteps
Episode  465:  Won after  36 timesteps
Episode  466: Lost after 200 timesteps
Episode  467:  Won after  69 timesteps
Episode  468:  Won after  84 timesteps
Episode  469: Lost after 200 timesteps
Episode  470:  Won after 137 timesteps
Episode  471:  Won after 161 timesteps
Episode  472:  Won after 142 timesteps
Episode  473:  Won after  87 timesteps
Episode  474:  Won after  50 timesteps
Episode  475:  Won after  53 timesteps
Episode  476:  Won after  75 timesteps
Episode  477:  Won after 110 timesteps
Episode  478:  Won after 115 timesteps
Episode  479:  Won after  55 timesteps
Episode  480:  Won after  37 timesteps
Episode  481: Lost after 200 timesteps
Episode  482:  Won after 

Episode  719:  Won after 158 timesteps
Episode  720:  Won after  72 timesteps
Episode  721:  Won after  93 timesteps
Episode  722:  Won after  54 timesteps
Episode  723:  Won after  91 timesteps
Episode  724: Lost after 200 timesteps
Episode  725:  Won after  68 timesteps
Episode  726:  Won after 107 timesteps
Episode  727: Lost after 200 timesteps
Episode  728:  Won after 100 timesteps
Episode  729: Lost after 200 timesteps
Episode  730:  Won after  96 timesteps
Episode  731:  Won after  37 timesteps
Episode  732: Lost after 200 timesteps
Episode  733:  Won after  95 timesteps
Episode  734: Lost after 200 timesteps
Episode  735:  Won after 124 timesteps
Episode  736:  Won after 165 timesteps
Episode  737: Lost after 200 timesteps
Episode  738:  Won after 104 timesteps
Episode  739:  Won after 146 timesteps
Episode  740:  Won after  80 timesteps
Episode  741:  Won after  60 timesteps
Episode  742:  Won after  63 timesteps
Episode  743:  Won after 108 timesteps
Episode  744:  Won after 

Episode  979:  Won after  53 timesteps
Episode  980:  Won after 168 timesteps
Episode  981:  Won after  45 timesteps
Episode  982:  Won after  72 timesteps
Episode  983:  Won after  67 timesteps
Episode  984:  Won after 147 timesteps
Episode  985: Lost after 200 timesteps
Episode  986:  Won after  36 timesteps
Episode  987: Lost after 200 timesteps
Episode  988:  Won after 137 timesteps
Episode  989:  Won after  50 timesteps
Episode  990:  Won after 123 timesteps
Episode  991:  Won after  74 timesteps
Episode  992: Lost after 200 timesteps
Episode  993:  Won after 189 timesteps
Episode  994:  Won after 172 timesteps
Episode  995:  Won after 123 timesteps
Episode  996:  Won after 116 timesteps
Episode  997:  Won after  40 timesteps
Episode  998:  Won after 188 timesteps
Episode  999:  Won after  82 timesteps
Episode 1000:  Won after 102 timesteps

Average reward: 0.9
